In [368]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import stats

In [370]:
df = pd.read_csv("MROZ.csv")

In [372]:
df.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,...,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,...,16310,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,...,21800,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,...,21040,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,...,7300,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,...,27300,0.6215,12,14,9.5,1,7,20.100058,1.524272,49


In [374]:
df = df.dropna()

Question:1

In [377]:
df['log_wage'] = np.log(df['wage'])
X1 = df[['educ', 'exper', 'expersq', 'kidslt6', 'kidsge6']]
X1 = sm.add_constant(X1)
y1 = df['log_wage']

In [379]:
print('Column names:', df.columns)

Column names: Index(['inlf', 'hours', 'kidslt6', 'kidsge6', 'age', 'educ', 'wage', 'repwage',
       'hushrs', 'husage', 'huseduc', 'huswage', 'faminc', 'mtr', 'motheduc',
       'fatheduc', 'unem', 'city', 'exper', 'nwifeinc', 'lwage', 'expersq',
       'log_wage'],
      dtype='object')


In [381]:
# calculating coefficients
beta_hat1 = np.linalg.inv(X1.T @ X1) @ X1.T @ y1
print('Coefficients (Stage I Regression):')
print(beta_hat1)

Coefficients (Stage I Regression):
0   -0.485650
1    0.107962
2    0.040034
3   -0.000803
4   -0.053418
5   -0.012470
dtype: float64


Question:2

In [384]:
residuals1 = y1.values - X1.values @ beta_hat1
sigma_squared1 = (residuals1.T @ residuals1) / (len(y1) - len(beta_hat1))
var_beta1 = sigma_squared1 * np.linalg.inv(X1.T @ X1)
std_errors1 = np.sqrt(np.diag(var_beta1))
print('Standard Errors (Stage I Regression):')
print(std_errors1)

Standard Errors (Stage I Regression):
[0.21364314 0.01437862 0.01335577 0.00039405 0.08467129 0.02679583]


Question:3

In [387]:
X2 = df[['exper', 'expersq', 'kidslt6', 'kidsge6', 'motheduc', 'fatheduc']]
X2 = sm.add_constant(X2)
y2 = df['educ']

# Manually calculate coefficients
alpha_hat = np.linalg.inv(X2.T @ X2) @ (X2.T @ y2)
print('Coefficients (Instrumental Variables Regression):')
print(alpha_hat)

Coefficients (Instrumental Variables Regression):
[ 9.32486193e+00  4.29398398e-02 -1.06809389e-03  6.52589952e-01
 -1.52581236e-01  1.60245239e-01  1.79623914e-01]


Question:4

In [390]:
residuals2 = y2.values - X2.values @ alpha_hat
sigma_squared2 = (residuals2.T @ residuals2) / (len(y2) - len(alpha_hat))
var_alpha = sigma_squared2 * np.linalg.inv(X2.T @ X2)
std_errors2 = np.sqrt(np.diag(var_alpha))
print('Standard Errors (Instrumental Variables Regression):')
print(std_errors2)

Standard Errors (Instrumental Variables Regression):
[0.46736174 0.04037097 0.00119265 0.25429651 0.08123817 0.03566071
 0.03371729]


Question:5

In [393]:
from scipy.stats import f
R = np.array([[0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1]])
r = np.zeros(2)
F_stat = ((R @ alpha_hat - r).T @ np.linalg.inv(R @ var_alpha @ R.T) @ (R @ alpha_hat - r)) / len(r)
df1, df2 = len(r), len(y2) - len(alpha_hat)
p_value = 1 - f.cdf(F_stat, df1, df2)
print('Joint Hypothesis Test (H0: α5 = 0, α6 = 0):')
print('F-statistic:', F_stat)
print('p-value:', p_value)



Joint Hypothesis Test (H0: α5 = 0, α6 = 0):
F-statistic: 53.520792596708475
p-value: 1.1102230246251565e-16


Question:6

In [396]:
df['educ_fitted'] = X2 @ alpha_hat

X3 = df[['educ_fitted', 'exper', 'expersq', 'kidslt6', 'kidsge6']]
X3 = sm.add_constant(X3)
y3 = df['log_wage']

beta_hat2 = np.linalg.inv(X3.T @ X3) @ (X3.T @ y3)
print('Coefficients (Stage II Regression):')
print(beta_hat2)

Coefficients (Stage II Regression):
[ 0.12246335  0.05971665  0.042541   -0.00089684 -0.01549278 -0.02234876]


Question 7

In [399]:
residuals3 = y3.values - X3.values @ beta_hat2
sigma_squared3 = (residuals3.T @ residuals3) / (len(y3) - len(beta_hat2))
var_beta2 = sigma_squared3 * np.linalg.inv(X3.T @ X3)
std_errors3 = np.sqrt(np.diag(var_beta2))
print('Standard Errors (Stage II Regression):')
print(std_errors3)


Standard Errors (Stage II Regression):
[4.43551801e-01 3.38775130e-02 1.42548182e-02 4.22140409e-04
 9.29141353e-02 2.90922601e-02]


Question:8

In [402]:
# Initilly i had done var_beta1[1, 1] - var_beta2[1, 1] i was getting 'Exogeneity test not computed due to negative variance difference.' Then i reversed 
# this and make var_beta2[1, 1] - var_beta1[1, 1]
var_diff = var_beta2[1, 1] - var_beta1[1, 1]
if var_diff > 0:
    H_stat = (beta_hat1[1] - beta_hat2[1]) / np.sqrt(var_diff)
    print('Exogeneity Test Statistic (H):', H_stat)

    if np.abs(H_stat) > 1.96:
        print('Reject H0: Education is exogenous.')
    else:
        print('Fail to reject H0: Education is exogenous.')


Exogeneity Test Statistic (H): 1.572812913582322
Fail to reject H0: Education is exogenous.
